### Imports

In [28]:
# Import packages
from llama_index import VectorStoreIndex
from llama_index import SimpleDirectoryReader
import logging
import sys
from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser
from oepul_chat.readers.custom_pdf_reader import CustomPDFReader
from oepul_chat.readers.custom_html_reader import CustomHTMLReader
from llama_index import SimpleDirectoryReader
import random
from llama_index.schema import MetadataMode
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext
from llama_index.llms import LangChainLLM
from llama_index import download_loader

PDFReader = download_loader("PDFReader")


# Import local library
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

# Autoreload local library
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Logging setup

In [3]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Data


### Data preprocessing, structure extraction
I built a CustomPDFReader which can be found in `oepul_chat/custom_pdf_reader.py` it extracts the structure out of the PDF and embeds it in the metadata field `header_path` i think this is one od the first crucial steps as it gives each text element a context in all of the files. With llama index we can then give this fiel to the retriever or the LLM or both. 

In [31]:
def load_data(filepath, filetype, reader):
    """Load markdown docs from a directory, excluding all other file types."""
    loader = SimpleDirectoryReader(
        input_dir=filepath,
        file_extractor={filetype: reader},
        recursive=True
    )

    data = loader.load_data()

    # print short summary
    print("Loaded {} documents".format(len(data)))
    print("First document metadata: {}".format(data[0].metadata))
    print("First document text: {}".format(data[0].text[0:80]))

    return data

# for testing only one doc 
oepul_official_docs = load_data("data/OEPUL_PDF/", ".pdf", CustomPDFReader())

Loaded 1104 documents
First document metadata: {'File Name': 'O6_10_Erosionsschutz_Wein_Obst_Hopfen_2022_12.pdf', 'Content Type': 'text', 'Header Path': 'Erosionsschutz Wein, Obst und Hopfen/K-Ö'}
First document text: 


In [32]:
bio_austria_guide = load_data("data/BIO_AUSTRIA", ".html", CustomHTMLReader())

Loaded 38 documents
First document metadata: {'File Name': 'aktueller-planungsstand-zu-bio-im-oepul-2023.html', 'Content Type': 'text', 'Header Path': 'Bio-Maßnahme/Bio-Basisprämie:/Auflagen Bio-Basisprämie:/Anlage von Biodiversitäts-Flächen/Vier Möglichkeiten im Grünland:/Einzuhalten bei Biodiversitäts-Flächen am Acker:/Optionale Module (einjährig):/Acker/Grünland:/N2000/WRRL(Wasserrahmenrichtlinie):/Beitrag teilen/Bio-Maßnahme/Bio-Basisprämie:/Auflagen Bio-Basisprämie:/Anlage von Biodiversitäts-Flächen/Vier Möglichkeiten im Grünland:/Einzuhalten bei Biodiversitäts-Flächen am Acker:/Optionale Module (einjährig):/Acker/Grünland:/N2000/WRRL(Wasserrahmenrichtlinie):/Beitrag teilen/Bio-Maßnahme/Bio-Basisprämie:/Auflagen Bio-Basisprämie:/Anlage von Biodiversitäts-Flächen/Vier Möglichkeiten im Grünland:/Einzuhalten bei Biodiversitäts-Flächen am Acker:/Optionale Module (einjährig):/Acker/Grünland:/N2000/WRRL(Wasserrahmenrichtlinie):/Beitrag teilen/Bio-Maßnahme/Bio-Basisprämie:/Auflagen Bio-B

In [33]:
# Load rest of docs with simple pdf reader
ama_official_docs = load_data("data/AMA", ".pdf", PDFReader())

Loaded 148 documents
First document metadata: {'page_label': '1', 'file_name': '20230131_Merkblatt_MFA2023_V3.pdf'}
First document text:  
 
MERKBLATT  
 
Mehrfachantrag 2023  
STAND 31. Jänner 2023   
(Version 3) 
 
 
 
 
 
 
 
Zertifiziertes Qualitätsmanagement -System nach 


In [35]:
# Show some example
print(random.sample(oepul_official_docs, 1)[0].get_content(
    metadata_mode=MetadataMode.ALL))

File Name: O6_11_Herbizidverzicht_Wein_Obst_Hopfen_2022_12.pdf
Content Type: text
Header Path: Herbizidverzicht Wein, Obst und Hopfen/BEANTRAGUNG

Folgende Punkte sind bei der Beantragung zu beachten: • Die Maßnahme „Herbizidverzicht Wein, Obst und Hopfen“ muss vor Verpflichtungsbeginn im Maßnahmenantrag des Mehrfachantrages bis spätestens am 31. Dezember beantragt werden, um eine gültige Verpflichtung ab dem Folgejahr am Betrieb zu begründen. • Der letzte Einstieg in die Maßnahme ist mit dem Förderjahr 2025 möglich (Beantragung bis spätestens am 31. Dezember 2024). • Während der Laufzeit der Maßnahme kann bis spätestens am 31. Dezember 2025 in die Maßnahme „Biologische Wirtschaftsweise“ umgestiegen werden. • Die Prämie für Obst wird nur für Anlagen mit qualitativ hochwertigem Pflanzgut gewährt. Daher ist grundsätzlich nur veredeltes Material zulässig. Dies gilt sowohl für Neuanlagen als auch für Altanlagen. Beispielsweise müssen unveredelte Walnussbäume (kein qualitativ hochwertiges P

Llama index support different content outputs per Document or Node depending on if the retriever or the LLM is accessing the document. For now we will exlude the file name as its mostly duplicate information.

In [37]:
# Hide the File Name from the LLM
for doc in oepul_official_docs:
    doc.excluded_llm_metadata_keys = ["File Name"]
    # doc.excluded_embed_metadata_keys = ["File Name"]


print("# Embed example")
print(random.sample(oepul_official_docs, 1)[0].get_content(
    metadata_mode=MetadataMode.EMBED))

print("\n# LLM example")
print(random.sample(oepul_official_docs, 1)[0].get_content(
    metadata_mode=MetadataMode.LLM))

# Embed example
File Name: O6_11_Herbizidverzicht_Wein_Obst_Hopfen_2022_12.pdf
Content Type: text
Header Path: Herbizidverzicht Wein, Obst und Hopfen/BEANTRAGUNG

Folgende Punkte sind bei der Beantragung zu beachten: • Die Maßnahme „Herbizidverzicht Wein, Obst und Hopfen“ muss vor Verpflichtungsbeginn im Maßnahmenantrag des Mehrfachantrages bis spätestens am 31. Dezember beantragt werden, um eine gültige Verpflichtung ab dem Folgejahr am Betrieb zu begründen. • Der letzte Einstieg in die Maßnahme ist mit dem Förderjahr 2025 möglich (Beantragung bis spätestens am 31. Dezember 2024). • Während der Laufzeit der Maßnahme kann bis spätestens am 31. Dezember 2025 in die Maßnahme „Biologische Wirtschaftsweise“ umgestiegen werden. • Die Prämie für Obst wird nur für Anlagen mit qualitativ hochwertigem Pflanzgut gewährt. Daher ist grundsätzlich nur veredeltes Material zulässig. Dies gilt sowohl für Neuanlagen als auch für Altanlagen. Beispielsweise müssen unveredelte Walnussbäume (kein qualitati

### Further splitting into Chunks
So far the documents have been splitted logically in hierarchies but we want them splitted in chunks this will be done with the Llama index SimpleNodeParser Object. The chunk_size and chunk overlap can be seen as tuning parameters.

In [6]:
node_parser = SimpleNodeParser.from_defaults(
    text_splitter=TokenTextSplitter(chunk_size=1024, chunk_overlap=20)
)

### Defining the embedding model

For testing i will use free Embeddings for document retrieval which will later be switched to OpenAI Embeddings.

In [7]:
# embed_model = OpenAIEmbedding()
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base"))

/Users/vali/anaconda3/envs/oepul-chat-m/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-base
Load pretrained SentenceTransformer: intfloat/multilingual-e5-base
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu
Use pytorch device: cpu


## LLM Pipeline 

Similar to the embeddings for testing a local LLM will be used.

### LLM definition

In [8]:
llm = OpenAI(model='text-davinci-003', temperature=0, max_tokens=256)
# from langchain.llms import LlamaCpp
# from langchain.callbacks.manager import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# # Callbacks support token-wise streaming
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# llm = LangChainLLM(llm=LlamaCpp(
#     model_path="/Users/vali/repos/models/openorca-platypus2-13b.ggmlv3.q4_0.bin",
#     temperature=0.75,
#     max_tokens=2000,
#     top_p=1,
#     callback_manager=callback_manager,
#     verbose=True,  # Verbose is required to pass to the callback manager
# ))

# from llama_index.llms import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

# llm = LlamaCPP(
#     # You can pass in the URL to a GGML model to download it automatically
#     model_url=model_url,
#     # optionally, you can set the path to a pre-downloaded model instead of model_url
#     model_path=None,
#     temperature=0.1,
#     max_new_tokens=256,
#     # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
#     context_window=3900,
#     # kwargs to pass to __call__()
#     generate_kwargs={},
#     # kwargs to pass to __init__()
#     # set to at least 1 to use GPU
#     model_kwargs={"n_gpu_layers": 1},
#     verbose=True,
# )


### Prompt and Pipeline

In [9]:
prompt_helper = PromptHelper(
    context_window=4096,
    num_output=256,
    chunk_overlap_ratio=0.1,
    chunk_size_limit=None
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
    prompt_helper=prompt_helper
)

## Try out the pipeline

In [11]:

query1 = "Welche Begrünungskulturen sind gültig?"
query2 = "Welche Maßnahme trägt zur Verringerung der Treibhausgasemission bei?"
query3 = "Was umfasst die Prämie Tierwohl Weide?"
query4 = "Welche spezifischen Maßnahmen werden im Rahmen des Grundwasserschutzprogramms Graz bis Bad Radkersburg ergriffen, um sicherzustellen, dass die Bodennutzung eine geringfügige Auswirkung auf das Grundwasser hat, und wie werden die Landwirte dabei unterstützt, die vorgegebenen jahreswirksamen Stickstoffdüngermengen pro Hektar und Jahr einzuhalten?"

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(service_context=service_context)

response = query_engine.query(query4)
response

Response(response='\nIm Rahmen des Grundwasserschutzprogramms Graz bis Bad Radkersburg werden spezifische Maßnahmen ergriffen, um sicherzustellen, dass die Bodennutzung eine geringfügige Auswirkung auf das Grundwasser hat. Dazu gehört, dass die maximal zulässigen jahreswirksamen Stickstoffdüngermengen pro Hektar und Jahr für die jeweilige Düngeklasse gemäß Anlage 3, Punkt 1 und 2 des Grundwasserschutzprogramms Graz bis Bad Radkersburg in Verbindung mit der Düngeklasseneinstufung in Anlage 2B derselben Verordnung eingehalten werden müssen. Um die Landwirte bei der Einhaltung dieser Vorgaben zu unterstützen, werden im Layer des Landes Steiermark, der im INVEKOS GIS einsehbar ist, die D', source_nodes=[NodeWithScore(node=TextNode(id_='95adad2d-ef36-42e2-a05c-a85d395a5f97', embedding=None, metadata={'File Name': 'O6_24_Wasserrahmenrichtlinie-Landwirtschaft_2022_12.pdf', 'Content Type': 'text', 'Header Path': 'Wasserrahmenrichtlinie – Landwirtschaft/FÖRDERBEDINGUNGEN/STICKSTOFF-DÜNGUNG'}, e

In [13]:
print(response)


Im Rahmen des Grundwasserschutzprogramms Graz bis Bad Radkersburg werden spezifische Maßnahmen ergriffen, um sicherzustellen, dass die Bodennutzung eine geringfügige Auswirkung auf das Grundwasser hat. Dazu gehört, dass die maximal zulässigen jahreswirksamen Stickstoffdüngermengen pro Hektar und Jahr für die jeweilige Düngeklasse gemäß Anlage 3, Punkt 1 und 2 des Grundwasserschutzprogramms Graz bis Bad Radkersburg in Verbindung mit der Düngeklasseneinstufung in Anlage 2B derselben Verordnung eingehalten werden müssen. Um die Landwirte bei der Einhaltung dieser Vorgaben zu unterstützen, werden im Layer des Landes Steiermark, der im INVEKOS GIS einsehbar ist, die D
